In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, BatchNormalization

# Step 1: Load stock market data
start = dt.datetime(2010, 1, 1)
end = dt.datetime.today()

df = yf.download(tickers=['^GSPC'], start=start, end=end, auto_adjust=True)
df = df.reset_index()

# Drop unnecessary columns
columns_to_drop = ['Date']
if 'Adj Close' in df.columns:
    columns_to_drop.append('Adj Close')
df = df.drop(columns=columns_to_drop)

# Step 2: Train-test split (70% train, 30% test)
train_size = int(len(df) * 0.70)
data_training = df['Close'][:train_size].values.reshape(-1, 1)
data_testing = df['Close'][train_size:].values.reshape(-1, 1)

# Step 3: Normalize data
scaler = MinMaxScaler(feature_range=(0,1))
data_training_scaled = scaler.fit_transform(data_training)

# Step 4: Prepare Training Sequences
X_train, y_train = [], []
sequence_length = 100  # Lookback window

for i in range(sequence_length, len(data_training_scaled) - 365):
    X_train.append(data_training_scaled[i-sequence_length:i])

    # Multi-horizon targets (future prices)
    y_train.append([
        data_training_scaled[i + 1, 0],   # 1 day ahead
        data_training_scaled[i + 5, 0],   # 5 days ahead
        data_training_scaled[i + 30, 0],  # 1 month ahead
        data_training_scaled[i + 365, 0]  # 1 year ahead
    ])

X_train, y_train = np.array(X_train), np.array(y_train)

print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)  # Should have 4 outputs per sample

# Step 5: Define Multi-Horizon LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, recurrent_dropout=0.2, input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.2),

    LSTM(80, return_sequences=True, recurrent_dropout=0.2),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(100, return_sequences=False, recurrent_dropout=0.2),
    BatchNormalization(),
    Dropout(0.3),

    Dense(4, activation="linear")  # 4 outputs for multi-horizon predictions
])

# Step 6: Compile the Model
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

# Step 7: Train the Model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Step 8: Save the Model
model.save("stock_model_multihorizon.h5")
print("Model training complete and saved as stock_model_multihorizon.h5")


[*********************100%***********************]  1 of 1 completed
C:\Users\Gourish\AppData\Local\Temp\ipykernel_23468\3723710596.py:21: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=columns_to_drop)
c:\Users\Gourish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


X_train Shape: (2211, 100, 1)
y_train Shape: (2211, 4)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 64)        │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 80)        │        46,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 80)        │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        72,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137,076 (535.45 KB)

 Trainable params: 136,588 (533.55 KB)

 Non-trainable params: 488 (1.91 KB)

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 77ms/step - loss: 1.2794 - mae: 0.8864 - val_loss: 0.2715 - val_mae: 0.4977
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.6369 - mae: 0.6312 - val_loss: 0.1470 - val_mae: 0.3558
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.4661 - mae: 0.5384 - val_loss: 0.1769 - val_mae: 0.3969
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.3631 - mae: 0.4799 - val_loss: 0.1253 - val_mae: 0.3416
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.2884 - mae: 0.4198 - val_loss: 0.1045 - val_mae: 0.3162
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.2186 - mae: 0.3646 - val_loss: 0.1132 - val_mae: 0.3269
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.1671 - mae: 0.3205 - val_loss: 0.0922 - val_mae: 0.2944
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.1283 - mae: 0.2818 - val_loss: 0.0472 - val_mae: 0.2034
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - lo

Model training complete and saved as stock_model_multihorizon.h5


In [1]:
from keras.models import load_model
import datetime as dt

# Load the saved model
model = load_model(r"C:\Users\Gourish\Desktop\stockpredictor\models\stock_model_multihorizon_keras.keras")
print("✅ Model loaded successfully.")

# Reimport necessary libraries
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

# Reload the stock market data to reinitialize scaler
start = dt.datetime(2010, 1, 1)
end = dt.datetime.today()

df = yf.download(tickers=['^GSPC'], start=start, end=end, auto_adjust=True)
df = df.reset_index()
print(df)
# Drop unnecessary columns
columns_to_drop = ['Date']
if 'Adj Close' in df.columns:
    columns_to_drop.append('Adj Close')
df = df.drop(columns=columns_to_drop)

# Train-test split
train_size = int(len(df) * 0.70)
data_training = df['Close'][:train_size].values.reshape(-1, 1)
data_testing = df['Close'][train_size:].values.reshape(-1, 1)

# Reinitialize and fit MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data_training_scaled = scaler.fit_transform(data_training)  # Fit on training data
data_testing_scaled = scaler.transform(data_testing)  # Transform test data

print("✅ Scaler reinitialized and test data transformed.")

# Create test sequences (same logic as training)
X_test, y_test = [], []
sequence_length = 100  # Lookback window

for i in range(sequence_length, len(data_testing_scaled) - 365):
    X_test.append(data_testing_scaled[i-sequence_length:i])
    y_test.append([
        data_testing_scaled[i + 1, 0],   # 1 day ahead
        data_testing_scaled[i + 5, 0],   # 5 days ahead
        data_testing_scaled[i + 30, 0],  # 1 month ahead
        data_testing_scaled[i + 365, 0]  # 1 year ahead
    ])

X_test, y_test = np.array(X_test), np.array(y_test)

print("✅ Test data prepared.")
print("X_test Shape:", X_test.shape)
print("y_test Shape:", y_test.shape)

# Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)

# Predict the test set
y_pred_scaled = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_real = scaler.inverse_transform(y_test)

# Compute error metrics
mse = np.mean((y_test_real - y_pred) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(abs(y_test_real - y_pred))
mape = mean_absolute_percentage_error(y_test_real, y_pred) * 100
# Print performance metrics
print("\n📊 Model Performance Metrics on Test Data:")
print(f"🔹 Mean Squared Error (MSE): {mse:.4f}")
print(f"🔹 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"🔹 Mean Absolute Error (MAE): {mae:.4f}")
print(f"🔹 Mean Absolute Percentage Error (MAPE): : {mape:.4f}") 

c:\Users\Gourish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 19 variables whereas the saved optimizer has 36 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


✅ Model loaded successfully.


[*********************100%***********************]  1 of 1 completed
C:\Users\Gourish\AppData\Local\Temp\ipykernel_20464\2306985004.py:25: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=columns_to_drop)


Price        Date        Close         High          Low         Open  \
Ticker                   ^GSPC        ^GSPC        ^GSPC        ^GSPC   
0      2010-01-04  1132.989990  1133.869995  1116.560059  1116.560059   
1      2010-01-05  1136.520020  1136.630005  1129.660034  1132.660034   
2      2010-01-06  1137.140015  1139.189941  1133.949951  1135.709961   
3      2010-01-07  1141.689941  1142.459961  1131.319946  1136.270020   
4      2010-01-08  1144.979980  1145.390015  1136.219971  1140.520020   
...           ...          ...          ...          ...          ...   
3852   2025-04-28  5528.750000  5553.660156  5468.640137  5529.220215   
3853   2025-04-29  5560.830078  5571.950195  5505.700195  5508.870117   
3854   2025-04-30  5569.060059  5581.839844  5433.240234  5499.439941   
3855   2025-05-01  5604.140137  5658.910156  5597.350098  5625.140137   
3856   2025-05-02  5686.669922  5700.700195  5642.279785  5645.879883   

Price       Volume  
Ticker       ^GSPC  
0       

In [3]:
model.save("stock_model_multihorizon_keras.keras")